# After refactoring...

In [5]:
import taichi as ti
import numpy as np

from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Initialize Taichi
ti.init(arch=ti.cpu)

# img_path = "/Users/balintl/Documents/GitHub/sonification/experiments/lsm_paper/figures/cellular_seq/Timepoint_001_220518-ST_C03_s1.jpg"
img_path = "test.jpg"

# Create app
app = App()
app.load_image(img_path) # Load image

# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

# Add objects to app
app.add_feature(mean_pix)
app.add_synth(theremin)
app.add_mapper(pix2freq)


[Taichi] Starting on arch=arm64


Canvas(height=580, width=1080)

IntSlider(value=50, description='Probe Width', max=500, min=1)

IntSlider(value=50, description='Probe Height', max=500, min=1)

ToggleButton(value=False, description='Audio', icon='speaker', tooltip='Toggle audio')

...

AudioGraph: The global audio graph has already been created. To create a new graph, call .destroy() first.


Text(value='', description='MeanPixelValue_0_Min:', disabled=True)

Text(value='', description='MeanPixelValue_0_Min:', disabled=True)

Text(value='', description='MeanPixelValue_0_Last:', disabled=True)

Button(description='Reset MeanPixelValue MinMax', style=ButtonStyle())

In [3]:
# test adding another mapping
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.add_mapper(pix2amp)

In [4]:
app.mappers

In [8]:
theremin["amplitude"]["buffer"].data[0][0]

np.float32(0.57557815)

In [5]:
import signalflow as sf
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)

 * audioout
   input0:
    * multiply
      input0:
       * bus
         input0:
          * stereo-panner
            input:
             * multiply
               input0:
                * sine
                  frequency:
                   * smooth
                     input:
                      * buffer-player
                        loop: 1.000000
                        rate: 1.000000
                     smooth: 0.999425
               input1:
                * smooth
                  input:
                   * buffer-player
                     loop: 1.000000
                     rate: 1.000000
                  smooth: 0.999425
            pan: 0.000000
      input1:
       * smooth
         input:
          * multiply
            input0:
             * decibels-to-amplitude
               input: 0.000000
            input1:
             * buffer-player
               loop: 1.000000
               rate: 1.000000
         smooth: 0.999425



In [4]:
# generate a test image and save to disk
import numpy as np
from PIL import Image

img = np.zeros((256, 256, 3), dtype=np.uint8)

for i in range(256):
    for j in range(256):
        img[i, j] = [i, j, 0]

img = Image.fromarray(img)
img.save("test.jpg")